# Ejercicios UD03_03

# Análisis de sentimientos con Distilbert

En la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb) hemos entrenado a Distilbert para clasificar los sentimientos en los textos, utilizando una base de datos de Tweets.

En este ejercicio, queremos usar Distilbert para clasificar las reseñas de películas y el mismo entrenamiento no nos sirve. Por esta razón, utilizaremos el aprendizaje de transferencia para adaptar el modelo a la nueva tarea, utilizando el `DataSet` de las reseñas de películas IMDB, disponibles en Huggingface como un `imdb`.

## Objetivos de la práctica

* Reproduce el proceso visto en la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb) para entrenar a Distilbert para clasificar los sentimientos en las reseñas de películas.
* Deberá entrenar el modelo (Distilbert u otros transformadores) con el `dataset` de las reseñas de películas IMDB.
* Deberá analizar las predicciones del modelo y compararlo con el modelo entrenado en la práctica [Analisis de sentimientos con DistilBERT](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/3.-models_llenguatge_ES.ipynb).
* Utilitza un `pipeline` de transformers para hacer predicciones [`zero-shot`](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.ZeroShotClassificationPipeline) sobre reseñas de películas. Comparar los resultados con los del modelo entrenado con el `dataset` de IMDB.
* Escribe una celda con tus comentarios y/o conclusiones finales comparando las dos alternativas realizadas en el ejercicio.

In [ ]:
# Instalamos las librerias que vamos a usar

import os
os.environ["WANDB_DISABLED"] = "true"

%pip install -U transformers datasets evaluate accelerate scikit-learn accuracy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import datasets

# Cargamos el Dataset
dataset = datasets.load_dataset('stanfordnlp/imdb')

# Mostramos los datos de ejemplo

dataset['train'][0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

### Tokenización

In [ ]:
# importamos el tokenizador de DistilBERT
from transformers import AutoTokenizer

# Cargamos el tokenizador
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

# Mostramos un ejemplo de tokenización
tokenizer.tokenize('FC Barcelona is fucked this year')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

['fc', 'barcelona', 'is', 'fucked', 'this', 'year']

In [ ]:
# Definimos una función para preprocesar el texto.
# Truncamos los textos para asegurarnos de que no excedan el máximo tamaño de entrada deDistilBert

def tokenize(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [ ]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Evaluación

In [ ]:
import evaluate

accuracy = evaluate.load('accuracy')

In [ ]:
# Definimos una función para calcular la precisión del modelo

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### Etiquetas

In [ ]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

### Fine tuning del modelo

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 1

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Definimos un **Training Arguments** con los parametros de entrenamiento

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### Entrenamiento del modelo

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dades_tokenitzades['train'],
    eval_dataset=dades_tokenitzades['test'],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.213500,0.191692,0.928400


TrainOutput(global_step=1563, training_loss=0.2726019000824033, metrics={'train_runtime': 1732.6916, 'train_samples_per_second': 14.428, 'train_steps_per_second': 0.902, 'total_flos': 3311921203200000.0, 'train_loss': 0.2726019000824033, 'epoch': 1.0})

### Inferencia

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
print(classifier("Suddenly, I'm not half the man I used to be, There's a shadow hanging over me, Oh, yesterday came suddenly"))
print(classifier("Don't stop me now. I'm havin' such a good time, I'm havin' a ball. If you wanna have a good time, just give me a call"))
print(classifier("Remember those who win the game. Lose the love they sought to gain. In debentures of quality. And dubious integrity. Their small town eyes will gape at you. In dull surprise when payment due. Exceeds accounts received. At seventeen"))
print(classifier("You got your bitches with the silicone injections. Crystal meth and yeast infections. Bleached blond hair, collagen lip injections. Who are you to criticize my intentions?. Got your subtle, manipulative devices. Just like you, I got my vices. I got a thought that would be nice. I'd like to crush your head, tight in my vice. Pain"))

Device set to use cuda:0


[{'label': 'SADNESS', 'score': 0.6291826367378235}]
[{'label': 'JOY', 'score': 0.5801544189453125}]
[{'label': 'SADNESS', 'score': 0.8905172944068909}]
[{'label': 'SADNESS', 'score': 0.9227145314216614}]


## Zero-shot

In [6]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

etiquetas = ["SADNESS","JOY","LOVE","ANGER","FEAR","SURPRISE"]

frases = [
    "Suddenly, I'm not half the man I used to be, There's a shadow hanging over me, Oh, yesterday came suddenly",
    "Don't stop me now. I'm havin' such a good time, I'm havin' a ball. If you wanna have a good time, just give me a call",
    "Remember those who win the game. Lose the love they sought to gain. In debentures of quality. And dubious integrity. Their small town eyes will gape at you. In dull surprise when payment due. Exceeds accounts received. At seventeen",
    "You got your bitches with the silicone injections. Crystal meth and yeast infections. Bleached blond hair, collagen lip injections. Who are you to criticize my intentions?. Got your subtle, manipulative devices. Just like you, I got my vices. I got a thought that would be nice. I'd like to crush your head, tight in my vice. Pain"
]

for frase in frases:
    resultado = classifier(frase, etiquetas)
    print(f"Texto: {resultado['sequence']}")
    print(f"Emoción Predicha: {resultado['labels'][0]} (confianza: {resultado['scores'][0]:.2f})")
    print("-" * 80)

Device set to use cuda:0


Texto: Suddenly, I'm not half the man I used to be, There's a shadow hanging over me, Oh, yesterday came suddenly
Emoción Predicha: SURPRISE (confianza: 0.77)
--------------------------------------------------------------------------------
Texto: Don't stop me now. I'm havin' such a good time, I'm havin' a ball. If you wanna have a good time, just give me a call
Emoción Predicha: JOY (confianza: 0.55)
--------------------------------------------------------------------------------
Texto: Remember those who win the game. Lose the love they sought to gain. In debentures of quality. And dubious integrity. Their small town eyes will gape at you. In dull surprise when payment due. Exceeds accounts received. At seventeen
Emoción Predicha: SURPRISE (confianza: 0.47)
--------------------------------------------------------------------------------
Texto: You got your bitches with the silicone injections. Crystal meth and yeast infections. Bleached blond hair, collagen lip injections. Who are yo

In [7]:
print(classifier(frases[3], etiquetas))

{'sequence': "You got your bitches with the silicone injections. Crystal meth and yeast infections. Bleached blond hair, collagen lip injections. Who are you to criticize my intentions?. Got your subtle, manipulative devices. Just like you, I got my vices. I got a thought that would be nice. I'd like to crush your head, tight in my vice. Pain", 'labels': ['SURPRISE', 'ANGER', 'SADNESS', 'FEAR', 'JOY', 'LOVE'], 'scores': [0.41540637612342834, 0.2715754210948944, 0.15093128383159637, 0.07038247585296631, 0.06830371171236038, 0.023400679230690002]}


## Conclusiones
Al comparar los modelos entrenados con el dataset de IMDB y el modelo zero-shot, se observan diferencias significativas en los resultados obtenidos.
Comparación de modelos:

Modelo entrenado con IMDB:

Resultados de inferencia:

Frase 1: Predicción: SADNESS (0.63)

Frase 2: Predicción: JOY (0.58)

Frase 3: Predicción: SADNESS (0.89)

Frase 4: Predicción: SADNESS (0.92)

Modelo zero-shot:

Resultados de inferencia:

Frase 1: Predicción: SURPRISE (0.77)

Frase 2: Predicción: JOY (0.55)

Frase 3: Predicción: SURPRISE (0.47)

Frase 4: Predicción: SURPRISE (0.42)

El modelo entrenado con IMDB tiende a clasificar las frases con una alta probabilidad de SADNESS en la mayoría de los casos. En cambio, el modelo zero-shot tiende mas a clasificarlas como SURPRISE.

cabe destacar la diferencia en la última frase, donde el modelo de IMDB predice SADNESS con un alto nivel de confianza (0.92), mientras que el modelo zero-shot la clasifica como SURPRISE con una confianza (0.42) y con un (0.27) de ANGER . Esto sugiere que el modelo de IMDB podría estar más sesgado hacia emociones negativas. Por los datos con los que se ha entrenado.


Conclusión final

A pesar de que el modelo de IMDB logra una buena precisión en entrenamiento y validación, su rendimiento en inferencia muestra un sesgo hacia emociones negativas, clasificando tres de las cuatro frases de prueba como SADNESS. En cambio el modelo zero-shot ofrece una confianza mas baja en la mayoria de casos.

El modelo zero-shot no está especializado en emociones, sino en entender relaciones generales entre frases, por lo que puede interpretar ciertos patrones en los textos de manera diferente a un modelo entrenado con datos específicos de sentimientos.

Por lo que zero-shot puede ser útil cuando no se tiene un modelo entrenado en un conjunto de datos específico, pero puede ser menos preciso que un modelo especializado.
